# CHARTEVENTS
CHARTEVENTS.csv is a 35GB file after uncompress. We first copy the first 1000 lines into a separate file for analsysiIt is split into 100,000 line chunks for easier uploading. The first chunk is loaded for analysis. 

In [1]:
# Uncompress data file
#!gunzip mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz

In [1]:
import os
import numpy as np
import pandas as pd

## Preview the contents of CHARTEVENTS
Create a sample file of the first 1000 line sof CHARTEVENTS.csv for analysis

In [3]:
# Create a file with the first 1000 lines of CHARTEVENTS.csv
!head -1000 "mimic-iii-clinical-database-1.4/xab" > "MIMIC-data/chart10000.csv"

In [2]:
# Load this first 1000 rows of CHARTEVENTS for analysis
# Reference the CHARTEVENTS column headers from https://mimic.physionet.org/mimictables/chartevents/

data_dir = "MIMIC-data"
df_charts = pd.read_csv(os.path.join(data_dir,"chart10000.csv"), header=None, names=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME',
       'STORETIME', 'CGID', 'VALUE', 'VALUENUM', 'VALUEUOM', 'WARNING',
       'ERROR', 'RESULTSTATUS', 'STOPPED'])

In [3]:
# EDA with pandas_profiling
from pandas_profiling import ProfileReport

# EDA report
profile_charts = ProfileReport(df_charts)
profile_charts.to_file(output_file='charts_profile.html')

In [5]:
df_charts.head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
172383,171734,1709,127294,207018,220277,2118-01-06 02:00:00,2118-01-06 02:36:00,21452,95.00,95.00,%,0,0,NaN,NaN
172384,171735,1709,127294,207018,220224,2118-01-06 02:39:00,2118-01-06 02:41:00,20889,77.00,77.00,mmHg,1,0,NaN,NaN
172385,171736,1709,127294,207018,220227,2118-01-06 02:39:00,2118-01-06 02:41:00,20889,95.00,95.00,%,0,0,NaN,NaN
172386,171737,1709,127294,207018,220235,2118-01-06 02:39:00,2118-01-06 02:41:00,20889,43.00,43.00,mmHg,0,0,NaN,NaN
172387,171738,1709,127294,207018,223830,2118-01-06 02:39:00,2118-01-06 02:41:00,20889,7.38,7.38,units,0,0,NaN,NaN
172388,171739,1709,127294,207018,224828,2118-01-06 02:39:00,2118-01-06 02:41:00,20889,0.00,0.00,mEq/L,0,0,NaN,NaN
172389,171740,1709,127294,207018,225698,2118-01-06 02:39:00,2118-01-06 02:41:00,20889,26.00,26.00,mEq/L,0,0,NaN,NaN
172390,171741,1709,127294,207018,220045,2118-01-06 03:00:00,2118-01-06 03:35:00,16818,76.00,76.00,bpm,0,0,NaN,NaN
172391,171742,1709,127294,207018,220050,2118-01-06 03:00:00,2118-01-06 03:35:00,16818,164.00,164.00,mmHg,0,0,NaN,NaN
172392,171743,1709,127294,207018,220051,2118-01-06 03:00:00,2118-01-06 03:35:00,16818,88.00,88.00,mmHg,0,0,NaN,NaN


### Observation:
The columns needed are HADM_ID, ITEMID and VALUENUM. HADM_ID will link to the ADMISSIONS table. ITEMID identifies the chart measure, which we need to cross-reference with D_ITEMS table. VALUENUM provides the numerical readings for each chart item. Only the rows containing the chart measures of interest should be retainedd. 

In [6]:
df_charts.groupby('ITEMID').count()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
ITEMID,,,,,,,,,,,,,,
220045,73,73,73,73,73,73,73,73,73,73,73,73,0,0
220046,9,9,9,9,9,9,9,9,9,9,9,9,0,0
220047,10,10,10,10,10,10,10,10,10,10,10,10,0,0
220050,24,24,24,24,24,24,24,24,24,24,24,24,0,0
220051,24,24,24,24,24,24,24,24,24,24,24,24,0,0
220052,25,25,25,25,25,25,25,25,25,25,25,25,0,0
220056,6,6,6,6,6,6,6,6,6,6,6,6,0,0
220058,6,6,6,6,6,6,6,6,6,6,6,6,0,0
220179,52,52,52,52,52,52,52,52,52,52,52,52,0,0


## Cross-referencing D_ITEMS.csv
To understand what each ITEMID represents, cross-reference the D_ITEMS.csv dictionary.


In [8]:
df_Ditems = pd.read_csv(os.path.join(data_dir,'D_ITEMS.csv'))
df_Ditems

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,PCA Lockout (Min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,PCA Medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,PCA Total Dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,PCV Exh Vt (Obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5,1157,927,Allergy 2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
6,1158,930,Ext,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
7,1159,935,Allergy 3,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
8,1160,938,blood cultures,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9,1161,940,trach care,NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [9]:
print(df_Ditems[df_Ditems['ITEMID']==220045]['LABEL'])
print(df_Ditems[df_Ditems['ITEMID']==220050]['LABEL'])
print(df_Ditems[df_Ditems['ITEMID']==220051]['LABEL'])
print(df_Ditems[df_Ditems['ITEMID']==223761]['LABEL'])
print(df_Ditems[df_Ditems['ITEMID']==220546]['LABEL'])


11498    Heart Rate
Name: LABEL, dtype: object
11502    Arterial Blood Pressure systolic
Name: LABEL, dtype: object
11503    Arterial Blood Pressure diastolic
Name: LABEL, dtype: object
12366    Temperature Fahrenheit
Name: LABEL, dtype: object
11472    WBC
Name: LABEL, dtype: object


In [10]:
# heart rate: ITEMID = 220045

df_charts[df_charts['ITEMID']==220045]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
172390,171741,1709,127294,207018,220045,2118-01-06 03:00:00,2118-01-06 03:35:00,16818,76.0,76.0,bpm,0,0,NaN,NaN
172434,170135,1709,127294,207018,220045,2118-01-02 22:15:00,2118-01-02 22:17:00,16302,79.0,79.0,bpm,0,0,NaN,NaN
172450,170151,1709,127294,207018,220045,2118-01-02 23:00:00,2118-01-02 23:31:00,16797,81.0,81.0,bpm,0,0,NaN,NaN
172497,171174,1709,127294,207018,220045,2118-01-05 00:45:00,2118-01-05 01:18:00,16707,92.0,92.0,bpm,0,0,NaN,NaN
172504,171181,1709,127294,207018,220045,2118-01-05 01:00:00,2118-01-05 01:17:00,16707,91.0,91.0,bpm,0,0,NaN,NaN
172517,171194,1709,127294,207018,220045,2118-01-05 02:00:00,2118-01-05 02:17:00,16707,86.0,86.0,bpm,0,0,NaN,NaN
172532,174658,1709,127294,207018,220045,2118-01-13 05:00:00,2118-01-13 05:13:00,21452,88.0,88.0,bpm,0,0,NaN,NaN
172569,175675,1709,127294,207018,220045,2118-01-15 11:00:00,2118-01-15 11:04:00,19589,54.0,54.0,bpm,0,0,NaN,NaN
172590,175696,1709,127294,207018,220045,2118-01-15 12:00:00,2118-01-15 12:28:00,19589,70.0,70.0,bpm,0,0,NaN,NaN
172600,175706,1709,127294,207018,220045,2118-01-15 13:00:00,2118-01-15 13:32:00,19589,71.0,71.0,bpm,0,0,NaN,NaN


In [11]:
# BP systolic: ITEMID = 220050
df_charts[df_charts['ITEMID']==220050].head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
172391,171742,1709,127294,207018,220050,2118-01-06 03:00:00,2118-01-06 03:35:00,16818,164.0,164.0,mmHg,0,0,NaN,NaN
172498,171175,1709,127294,207018,220050,2118-01-05 00:45:00,2118-01-05 01:18:00,16707,121.0,121.0,mmHg,0,0,NaN,NaN
172505,171182,1709,127294,207018,220050,2118-01-05 01:00:00,2118-01-05 01:17:00,16707,129.0,129.0,mmHg,0,0,NaN,NaN
172533,174659,1709,127294,207018,220050,2118-01-13 05:00:00,2118-01-13 05:13:00,21452,157.0,157.0,mmHg,0,0,NaN,NaN
172743,171762,1709,127294,207018,220050,2118-01-06 04:00:00,2118-01-06 04:13:00,21452,170.0,170.0,mmHg,0,0,NaN,NaN


In [12]:
# BP diastolic: ITEMID = 220051
df_charts[df_charts['ITEMID']==220051]['VALUENUM'].count()
df_charts[df_charts['ITEMID']==220051]['VALUENUM'].describe()

count     24.000000
mean      82.958333
std       10.780333
min       56.000000
25%       75.750000
50%       81.000000
75%       90.500000
max      101.000000
Name: VALUENUM, dtype: float64

In [13]:
# Temperature F: ITEMID = 223761
df_charts[df_charts['ITEMID']==223761].count()

ROW_ID          26
SUBJECT_ID      26
HADM_ID         26
ICUSTAY_ID      26
ITEMID          26
CHARTTIME       26
STORETIME       26
CGID            26
VALUE           26
VALUENUM        26
VALUEUOM        26
WARNING         26
ERROR           26
RESULTSTATUS     0
STOPPED          0
dtype: int64

In [14]:
# White blood count (WBC): ITEMID = 220546
df_charts[df_charts['ITEMID']==220546].head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
172459,170160,1709,127294,207018,220546,2118-01-02 23:07:00,2118-01-02 23:51:00,20889,6.6,6.6,K/uL,0,0,NaN,NaN
172540,174666,1709,127294,207018,220546,2118-01-13 05:12:00,2118-01-13 05:35:00,20889,12.3,12.3,K/uL,1,0,NaN,NaN
173224,172093,1709,127294,207018,220546,2118-01-06 22:24:00,2118-01-06 22:49:00,20889,6.8,6.8,K/uL,0,0,NaN,NaN
173316,175553,1709,127294,207018,220546,2118-01-15 03:00:00,2118-01-15 04:31:00,20889,8.9,8.9,K/uL,0,0,NaN,NaN


### Input selection from Chart events
The following chart events will be used as input:
* Heart rate: ITEMID = 220045
* BP systolic: ITEMID = 220050
* BP diastolic: ITEMID = 220051
* Temperature F: ITEMID = 223761
* White blood count (WBC): ITEMID = 220546

The item codes are retrieved from D_ITEMS.csv

## Extract chart features
Using awk command to filter for specific itemID through all the chunks



In [16]:
!awk -F, '$6 == "220045" {print $4, $11 } ' mimic-iii-clinical-database-1.4/x* > MIMIC-data/Heartrate_220045.csv
!awk -F, '$6 == "220050" {print $4, $11 } ' mimic-iii-clinical-database-1.4/x* > MIMIC-data/BPsys_220050.csv
!awk -F, '$6 == "220051" {print $4, $11 } ' mimic-iii-clinical-database-1.4/x* > MIMIC-data/BPdia_220051.csv
!awk -F, '$6 == "223761" {print $4, $11 } ' mimic-iii-clinical-database-1.4/x* > MIMIC-data/Temp_223761.csv
!awk -F, '$6 == "220546" {print $4, $11 } ' mimic-iii-clinical-database-1.4/x* > MIMIC-data/WBC_220546.csv


### Proceed to analysis and feature engineering next